In [2]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [30]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import InputLayer

In [4]:
import keras_tuner

In [5]:
train_obj = preprocess.pre_process(preprocess.train_df)
val_obj = preprocess.pre_process(preprocess.val_df)
test_obj = preprocess.pre_process(preprocess.test_df)

In [6]:
x_train = train_obj.preprocess()
x_train = train_obj.lemmatise()

x_val = val_obj.preprocess()
x_val = val_obj.lemmatise()

x_test = test_obj.preprocess()
x_test = test_obj.lemmatise()

In [7]:
y_train = train_obj.encode_class()
y_val = val_obj.encode_class()
y_test = test_obj.encode_class()

In [8]:
# vocab = list(set(" ".join(preprocess.pd.concat([x_train, x_val, x_test])).split()))

In [9]:
vocab_size = 20000
sequence_length = 25

In [11]:
textvector_layer = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="int", 
                                    #  pad_to_max_tokens=True)
                                    output_sequence_length=25)

In [12]:
textvector_layer.adapt(preprocess.pd.concat([x_train, x_val, x_test],
                                            ignore_index=True))

In [11]:
# textvector_layer.get_vocabulary()

In [12]:
# print(textvector_layer(x_train).numpy()[0])

In [13]:
hp = keras_tuner.HyperParameters()


In [15]:
class lstm():
    def lstm_model(self, dropout, dropout_rate, convolution, convolution2, 
                   filter, kernel_size, activation, pool_size, units):
        model = Sequential()
        model.add(textvector_layer)
        model.add(Embedding(input_dim=vocab_size+1, 
                            output_dim=sequence_length))
        
        if dropout:
            model.add(Dropout(rate=dropout_rate))
        else:
            model.add(SpatialDropout1D(rate=dropout_rate))
        
        if convolution:
            model.add(Conv1D(filters=filter, 
                             kernel_size=kernel_size, 
                             padding="same",
                             activation=activation))
            model.add(MaxPooling1D(pool_size=pool_size))
        
        if convolution2:
            model.add(Conv1D(filters=filter, 
                             kernel_size=kernel_size, 
                             padding="same",
                             activation=activation))
            model.add(MaxPooling1D(pool_size=pool_size))
        
        model.add(LSTM(units=units, 
                       dropout=dropout_rate,
                       recurrent_dropout=dropout_rate, 
                       activation=activation))
        

In [ ]:

def build_model(hp):
    model = Sequential()
    model.add(textvector_layer)
    model.add(Embedding(input_dim=vocab_size+1, 
                        output_dim=sequence_length))
    
    # dropout layer 1
    dropout_rate1 = hp.Float("dropout_rate1", 
                             min_value=0.2, 
                             max_value=0.8, 
                             step=0.025)
    if hp.Boolean("dropout1"):
            model.add(Dropout(rate=dropout_rate1))
    
    # convolutional layer 1
    if hp.Boolean("convolution1"):
        model.add(Conv1D(filters=hp.Int("filter1", 
                                            min_value=32, 
                                            max_value=512, 
                                            step=32), 
                            kernel_size=hp.Int("kernel_size1", 
                                            min_value=2, 
                                            max_value=8, 
                                            step=1), 
                            padding="valid",
                            activation="relu"))
        # if hp.Boolean("pooling1"):
        #     model.add(MaxPooling1D(pool_size=hp.Int("pool_size1", 
        #                                             min_value=2, 
        #                                             max_value=8, 
        #                                             step=1), 
        #                                             padding="valid"))
        # else:
        #     model.add(GlobalMaxPool1D())
    
    # dropout layer 2
    dropout_rate2 = hp.Float("dropout_rate2", 
                             min_value=0.2, 
                             max_value=0.8, 
                             step=0.025)
    if hp.Boolean("dropout2"):
         model.add(Dropout(rate=dropout_rate2))
    
    # convolutional layer 2
    if hp.Boolean("convolution2"):
        model.add(Conv1D(filters=hp.Int("filter2", 
                                            min_value=32, 
                                            max_value=512, 
                                            step=32), 
                            kernel_size=hp.Int("kernel_size2", 
                                            min_value=2, 
                                            max_value=8, 
                                            step=1), 
                            padding="valid",
                            activation="relu"))
        # if hp.Boolean("pooling2"):
        #     model.add(MaxPooling1D(pool_size=hp.Int("pool_size2", 
        #                                             min_value=2, 
        #                                             max_value=8, 
        #                                             step=1), padding="valid"))
        # else:
        #     model.add(GlobalMaxPool1D())

    if hp.Boolean("dropout3"):
            model.add(Dropout(rate=hp.Float("dropout_rate3", 
                                            min_value=0.2, 
                                            max_value=0.8, 
                                            step=0.025)))

    
    model.add(LSTM(units=hp.Int("lstm_units",
                                min_value=32, 
                                max_value=1024, 
                                step=32)))
    
    if hp.Boolean("dropout4"):
            model.add(Dropout(rate=hp.Float("dropout_rate4", 
                                            min_value=0.2, 
                                            max_value=0.8, 
                                            step=0.025)))
        
    model.add(Flatten())
    model.add(Dense(units=150,
                    activation="softmax"))
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    # model.compile(optimizer=hp.Choice("optimizer", 
    #                                   ["keras.optimizers.Adam(learning_rate=lr)",
    #                                    "keras.optimizers.Nadam(learning_rate=lr)",
    #                                    "keras.optimizers.SGD(learning_rate=lr)"]), 
    #               loss=hp.Choice("loss", ["keras.losses.CategoricalCrossentropy()",
    #                                       "keras.losses.KLDivergence()"]), 
    #               metrics=["accuracy", "val_accuracy"])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                    loss=keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy"])

    return model

In [60]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=3,
    overwrite=True,
    directory="lstm",
    project_name="lstm",
)

In [61]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
dropout_rate1 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.8, 'step': 0.025, 'sampling': 'linear'}
dropout1 (Boolean)
{'default': False, 'conditions': []}
convolution1 (Boolean)
{'default': False, 'conditions': []}
dropout2 (Boolean)
{'default': False, 'conditions': []}
convolution2 (Boolean)
{'default': False, 'conditions': []}
dropout3 (Boolean)
{'default': False, 'conditions': []}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
dropout4 (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [62]:
tuner.search(x_train[:3200], 
             y_train[:3200], 
             epochs=20, 
             validation_data=(x_val[:1500], y_val[:1500]))

Trial 4 Complete [00h 08m 57s]
val_accuracy: 0.09177777667840321

Best val_accuracy So Far: 0.18511111040910086
Total elapsed time: 00h 42m 30s


In [63]:
tuner.results_summary()

Results summary
Results in lstm\lstm
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
dropout_rate1: 0.375
dropout1: True
convolution1: False
dropout2: False
convolution2: False
dropout3: True
lstm_units: 32
dropout4: False
lr: 0.0037702081059086687
dropout_rate3: 0.2
Score: 0.18511111040910086

Trial 1 summary
Hyperparameters:
dropout_rate1: 0.275
dropout1: False
convolution1: False
dropout2: True
convolution2: False
dropout3: True
lstm_units: 896
dropout4: True
lr: 0.005265236635386028
dropout_rate3: 0.42500000000000004
dropout_rate4: 0.2
Score: 0.18488889187574387

Trial 3 summary
Hyperparameters:
dropout_rate1: 0.35000000000000003
dropout1: False
convolution1: False
dropout2: False
convolution2: False
dropout3: False
lstm_units: 384
dropout4: False
lr: 0.00019445820150205536
dropout_rate3: 0.5
dropout_rate4: 0.25
filter2: 128
kernel_size2: 8
Score: 0.09177777667840321

Trial 2 summary
Hyperparameters:
dropout_rate1: 0.625
drop

In [65]:
models = tuner.get_best_models(num_models=2)

c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [66]:
models[0].summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (32, 25)               │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (32, 25, 25)           │       500,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 25, 25)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (32, 25, 25)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 32)               │         7,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 32)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 150)              │         4,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,399 (1.95 MB)

 Trainable params: 512,399 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
models[1].summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (32, 25)               │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (32, 25, 25)           │       500,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 25, 25)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (32, 25, 25)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 896)              │     3,304,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (32, 896)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 896)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 150)              │       134,550 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,939,023 (15.03 MB)

 Trainable params: 3,939,023 (15.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
x_train.shape

(7500,)

In [ ]:
# model1 = Sequential()
# model1.add(textvector_layer)
# model1.add(Embedding(vocab_size+1, sequence_length))

# model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.02), 
#                       loss=keras.losses.CategoricalCrossentropy(),
#                       metrics=["accuracy"])

# epochs = 150
# batch_size = 64
# history = model1.fit(x_train, y_train,
#                     validation_data=(x_val, y_val),
#                     epochs=epochs, 
#                     batch_size=batch_size,
#                     callbacks=[EarlyStopping(monitor='val_accuracy', 
#                                              patience=15, 
#                                              min_delta=0.0001)])

In [50]:
model = Sequential()
model.add(textvector_layer)
model.add(Embedding(vocab_size+1, sequence_length))
# model.add(Conv1D(128, 2, activation="relu", padding="same"))
# model.add(MaxPooling1D())
model.add(Conv1D(32, 2, activation="relu", padding="same"))
model.add(MaxPooling1D())
model.add(LSTM(100, activation="tanh"))
# model.add(LSTM(50, activation="tanh"))
model.add(Flatten())
model.add(Dense(150, activation="softmax"))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.02), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

NameError: name 'textvector_layer' is not defined

In [ ]:
model2 = Sequential()
model2.add(textvector_layer)
model2.add(Embedding(vocab_size+1, sequence_length))
model2.add(Conv1D(64, kernel_size=3, activation="relu", padding="same"))
model2.add(Dropout(0.2))
model2.add(Conv1D(32, kernel_size=2, activation="relu", padding="same"))
model2.add(Dropout(0.2))
model2.add(LSTM(150, activation="tanh", return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(100, activation="tanh"))
model2.add(Dropout(0.2))

model2.add(Flatten())
model2.add(Dense(150, activation="softmax"))

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model2.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

Epoch 1/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 13s 66ms/step - accuracy: 0.0074 - loss: 4.9670 - val_accuracy: 0.0123 - val_loss: 4.5786
Epoch 2/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.0167 - loss: 4.4976 - val_accuracy: 0.0293 - val_loss: 4.3054
Epoch 3/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.0237 - loss: 4.2333 - val_accuracy: 0.0270 - val_loss: 4.1943
Epoch 4/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.0382 - loss: 3.9945 - val_accuracy: 0.0493 - val_loss: 4.0142
Epoch 5/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.0393 - loss: 3.8060 - val_accuracy: 0.0453 - val_loss: 3.9073
Epoch 6/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.0491 - loss: 3.6586 - val_accuracy: 0.0500 - val_loss: 3.8525
Epoch 7/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.0506 - loss: 3.5481 - val_accuracy: 0.0530 - val_loss: 3.7946
Epoch 8/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.0605 - loss: 3.4549 -

In [23]:
model2.evaluate(x_test, y_test, batch_size=64)

71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5026 - loss: 3.5022


[3.3088884353637695, 0.5291110873222351]

In [32]:
model2.predict(x_test[:2])[0]
preprocess.ohe.inverse_transform(model2.predict(x_test[:2]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


array([['what_song'],
       ['no']], dtype=object)

In [34]:
preprocess.ohe.inverse_transform(y_test[:2])

array([['translate'],
       ['translate']], dtype=object)

In [47]:
import numpy as np

In [53]:
np.argmax(model2.predict(x_test[:2])[1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


76

In [54]:
preprocess.ohe.categories_[0][76]

'no'

In [ ]:
class model():
    def __init__(self, hp, num_class, vocab_size, embedding_seq_length):
        self.hp = hp
        self.num_class = num_class 
        self.vocab_size = vocab_size
        self.embedding_seq_length = embedding_seq_length

    def base_layer(self):
        model = Sequential()
        model.add(textvector_layer)
        model.add(Embedding(self.vocab_size+2, 
                            self.embedding_seq_length, 
                            trainable="True"))

        return model
    
    def top_layer(self, model):
        model.add(Flatten())
        model.add(Dense(self.num_class, activation="softmax"))
    
    def lstm(self, lstm_units, rate, activation):
        model = self.base_layer()
        model.add(LSTM(lstm_units, 
                       dropout=rate,
                       recurrent_dropout=rate,
                       activation=activation))
        model.add(Flatten())
        self.top_layer(model=model)

        return model
    
    def stacked_lstm(self, lstm_units1, lstm_units2, 
                     lstm_units3, rate3, activation, 
                     rate1=0.2, rate2=0.2, lstm_layer=False):
        model = self.base_layer()

        model.add(LSTM(lstm_units1, 
                       activation=activation,
                       dropout=rate1,
                       recurrent_dropout=rate1, 
                       return_sequences=True))

        model.add(LSTM(lstm_units2, 
                       activation=activation,
                       dropout=rate2,
                       recurrent_dropout=rate2,
                       return_sequences=True if lstm_layer else False))
            
        if lstm_layer:
            model.add(LSTM(lstm_units3, 
                       activation=activation,
                       dropout=rate3,
                       recurrent_dropout=rate3))

        self.top_layer(model=model)

    def convo_lstm(self, convo_units, lstm_units, 
                   kernal_size, rate, convo_activaton):
        model = self.base_layer()

        model.add(Conv1D(units=convo_units, 
                         kernal_size=kernal_size, 
                         activation=convo_activaton, 
                         padding="same"))
        model.dropout(rate)

In [ ]:
model3 = Sequential()
model3.add(textvector_layer)
model3.add(Embedding(vocab_size+2, 128))
# model3.add(Conv1D(64, 
#                   kernel_size=3, 
#                   activation="relu", 
#                   padding="same"))
# model3.add(Dropout(0.2))
# model3.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model3.add(Dropout(0.2))
# model3.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model3.add(LSTM(100, 
                recurrent_dropout=0.2,
                activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model3.add(Flatten())
model3.add(Dense(150, activation="softmax"))

model3.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model3.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

Epoch 1/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.0056 - loss: 4.9976 - val_accuracy: 0.0160 - val_loss: 4.6170
Epoch 2/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.0229 - loss: 4.4653 - val_accuracy: 0.0530 - val_loss: 4.0403
Epoch 3/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - accuracy: 0.0570 - loss: 3.9135 - val_accuracy: 0.0800 - val_loss: 3.6655
Epoch 4/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.0848 - loss: 3.4475 - val_accuracy: 0.1080 - val_loss: 3.4023
Epoch 5/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.1072 - loss: 3.1518 - val_accuracy: 0.1510 - val_loss: 3.1932
Epoch 6/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.1650 - loss: 2.8599 - val_accuracy: 0.1867 - val_loss: 3.0470
Epoch 7/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.2040 - loss: 2.5904 - val_accuracy: 0.2137 - val_loss: 2.9015
Epoch 8/150
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.2665 - loss: 2.3683 -

In [26]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
x_train.values

array(['walk setting direct deposit bank internet saving account',
       'want switch direct deposit', 'set direct deposit', ...,
       'increase credit limit old navy card',
       'increase credit limit kohl card', 'increase credit limit dollar'],
      dtype=object)

In [49]:
train = tf.data.Dataset.from_tensor_slices((x_train.values, y_train))
val = tf.data.Dataset.from_tensor_slices((x_val.values, x_val))
test = tf.data.Dataset.from_tensor_slices((x_test.values, y_test))

In [28]:
train = train.shuffle(512).batch(64).prefetch(tf.data.AUTOTUNE).cache()
val = val.shuffle(512).batch(64).prefetch(tf.data.AUTOTUNE).cache()
test = test.batch(64).prefetch(tf.data.AUTOTUNE).cache()

In [13]:
vocab_size = 20000

In [ ]:
textvector_layer1 = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="int",
                                    #  pad_to_max_tokens=True)
                                    output_sequence_length=sequence_length)

In [13]:
textvector_layer1.adapt(preprocess.pd.concat([x_train, x_val],
                                            ignore_index=True))

In [17]:
y_train.shape

(7500, 150)

In [18]:
x_train.shape

(7500,)

In [41]:
model4 = Sequential()
# model4.add(InputLayer(shape=(None, None, None)))
model4.add(textvector_layer1)
model4.add(Embedding(vocab_size+2, 128))
# model3.add(Conv1D(64, 
#                   kernel_size=3, 
#                   activation="relu", 
#                   padding="same"))
# model3.add(Dropout(0.2))
# model3.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model3.add(Dropout(0.2))
# model3.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model4.add(LSTM(100, 
                dropout=0.4,
                recurrent_dropout=0.4,
                activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model4.add(Flatten())
model4.add(Dense(150, activation="softmax"))

model4.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 50
batch_size = 128

history = model4.fit(train,
                    validation_data=val,
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

Epoch 1/50
117/118 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0198 - loss: 4.8376

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 150)

In [47]:
y_train.reshape(1, 7500, 150)

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [13]:
for row in tf.data.Dataset.from_tensor_slices((x_train, y_train)).take(1):
    print(row)

(<tf.Tensor: shape=(), dtype=string, numpy=b'walk setting direct deposit bank internet saving account'>, <tf.Tensor: shape=(150,), dtype=float64, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>)
